<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/AdiabaticFlameTemperature_inclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/chetools.git

Cloning into 'chetools'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 72 (delta 47), reused 67 (delta 45), pack-reused 0
Unpacking objects: 100% (72/72), done.


In [2]:
import sys
sys.path.insert(1, "/content/chetools")
import numpy as np
from scipy.optimize import root
import tools.che as che

In [3]:
p = che.Props(['Methane', 'Ethane', 'nPropane',
               'Oxygen','Nitrogen', 'CarbonDioxide', 'Water'])

In [11]:
p.HfIG

array([ -74850.        ,  -83850.        , -104680.        ,
             0.        ,       0.        , -393720.98479714,
       -241820.        ])

In [4]:
def integrateCp(T):
    return (p.CpIGA*T + p.CpIGB*p.CpIGC/np.tanh(p.CpIGC/T) - p.CpIGD*p.CpIGE*np.tanh(p.CpIGE/T))/1000.

def H(T):
    return p.HfIG + integrateCp(T) - integrateCp(298.)

In [21]:
n_fuel = np.array([0.95, 0.04, 0.01]) # moles of methane, ethane, nPropane
excess_air = 0.2  #20% excess air

fuelCH = np.array([[1,4],
                   [2,6],
                   [3,8]])

nC, nH = n_fuel @ fuelCH

nCO2 = nC
nH2O = nH/2
nO2 = (nH2O/2 + nCO2)*(1.+excess_air)
nN2 = 79./21. * nO2

In [22]:
n_in= np.concatenate([n_fuel, np.array([nO2, nN2, 0., 0.])])
n_out = np.array([0., 0., 0., nO2*excess_air, nN2,nCO2, nH2O])

In [23]:
def eqs(vec):
    T = vec[0]
    return [np.dot(H(T), n_out) - np.dot(H(298), n_in)]

In [24]:
root(eqs, [2000.]).x[0]

2063.747638110465